In [84]:

from stellargraph import datasets, StellarGraph
from IPython.display import display, HTML
import networkx as nx
import pandas as pd
import pickle
import os
import glob

In [5]:
# !conda install -c stellargraph stellargraph -y

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: done

## Package Plan ##

  environment location: /Library/anaconda3

  added / updated specs:
    - stellargraph


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |              mkl           3 KB
    absl-py-0.9.0              |           py37_0         164 KB
    astor-0.8.0                |           py37_0          46 KB
    boto3-1.12.39              |             py_0          89 KB
    botocore-1.15.39           |             py_0         2.9 MB
    c-ares-1.15.0              |    h1de35cc_1001          73 KB
    cachetools-3.1.1           |             py_0          14 KB
    gast-0.3.3                 |             py_0          14 KB
    ge

## Metapath2vec using Stellargraph

In [37]:
user_artists = pd.read_csv('user_artist_cleaned.csv')

# nodes = pd.DataFrame([], index=["a", "b", "c", "d"])

# 	userID	artistID	weight	id	artist_name	cnt
# 0	2	51	13883	51	Duran Duran	111
# 1	4	51	228	51	Duran Duran	111
# 2	27	51	85	51	Duran Duran	111
# 3	28	51	10	51	Duran Duran	111

user_nodes = pd.DataFrame([], index=list(user_artists['userID'].unique()))
artist_nodes = pd.DataFrame([], index=list(user_artists['artist_name'].unique()))

edges = pd.DataFrame({
    "source": list(user_artists['userID']),
    "target": list(user_artists['artist_name']),
    "weight": list(user_artists['weight'])
})

Gs = StellarGraph({"user": user_nodes, "artist": artist_nodes}, edges)

In [83]:
from stellargraph.data import UniformRandomMetaPathWalk

rw = UniformRandomMetaPathWalk(Gs)

metapaths = [
#     ["user", "artist", "artist", "user"],
     ["user", "artist", "user"],
]

for walk_length in range(10,110,10):
    for N in range(5,25,5):
        walks = rw.run(
            nodes=list(Gs.nodes()),  # root nodes
            length=walk_length,  # maximum length of a random walk
            n=N,  # number of random walks per root node
            metapaths=metapaths,  # the metapaths
        )
        filename = "./data/metapaths/" + ''.join([node[0].upper() for node in metapaths[0]]) \
        + "/" + "walk_length_" + str(walk_length)+ "_" + str(N) +"_per_node" + ".p"
        print(filename)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        pickle.dump( walks, open( filename, "wb" ) )


./data/metapaths/UAU/walk_length_10_5_per_node.p
./data/metapaths/UAU/walk_length_10_10_per_node.p
./data/metapaths/UAU/walk_length_10_15_per_node.p
./data/metapaths/UAU/walk_length_10_20_per_node.p
./data/metapaths/UAU/walk_length_20_5_per_node.p
./data/metapaths/UAU/walk_length_20_10_per_node.p
./data/metapaths/UAU/walk_length_20_15_per_node.p
./data/metapaths/UAU/walk_length_20_20_per_node.p
./data/metapaths/UAU/walk_length_30_5_per_node.p
./data/metapaths/UAU/walk_length_30_10_per_node.p
./data/metapaths/UAU/walk_length_30_15_per_node.p
./data/metapaths/UAU/walk_length_30_20_per_node.p
./data/metapaths/UAU/walk_length_40_5_per_node.p
./data/metapaths/UAU/walk_length_40_10_per_node.p
./data/metapaths/UAU/walk_length_40_15_per_node.p
./data/metapaths/UAU/walk_length_40_20_per_node.p
./data/metapaths/UAU/walk_length_50_5_per_node.p
./data/metapaths/UAU/walk_length_50_10_per_node.p
./data/metapaths/UAU/walk_length_50_15_per_node.p
./data/metapaths/UAU/walk_length_50_20_per_node.p
./dat

In [139]:
pickle.load(open("./data/metapaths/UAU/walk_length_10_5_per_node.p", "rb"))

[[2,
  'Coldplay',
  1503,
  'David Bowie',
  142,
  'The Doors',
  1574,
  'Linkin Park',
  1553,
  'Pixie Lott'],
 [2,
  'Portishead',
  1688,
  'Portishead',
  2083,
  'Radiohead',
  1469,
  'The Strokes',
  1778,
  'Radiohead'],
 [2,
  'Madonna',
  21,
  'Kylie Minogue',
  1607,
  'Britney Spears',
  571,
  'Katy Perry',
  2040,
  'Ke$ha'],
 [2,
  'Duran Duran',
  1384,
  'Duran Duran',
  1541,
  'The Beatles',
  1206,
  'Iron Maiden',
  1707,
  'Oasis'],
 [2,
  'Depeche Mode',
  1327,
  'New Order',
  1746,
  'New Order',
  176,
  'The Beatles',
  1538,
  'System of a Down'],
 [4,
  'Radiohead',
  1795,
  'Arcade Fire',
  166,
  'The Killers',
  196,
  'Arctic Monkeys',
  821,
  'David Bowie'],
 [4,
  'Duran Duran',
  832,
  'Nightwish',
  929,
  'Coldplay',
  431,
  'blink-182',
  818,
  'MGMT'],
 [4,
  'Radiohead',
  773,
  'The White Stripes',
  913,
  'Death Cab for Cutie',
  1573,
  'Placebo',
  4,
  'Duran Duran'],
 [4,
  'Enrique Iglesias',
  638,
  'Avril Lavigne',
  240,


In [140]:
from gensim.models import Word2Vec, KeyedVectors

files = glob.glob("./data/metapaths/UAU/*.p")


for file in files:
    walks = pickle.load( open( file, "rb" ) )
    str_walks = [[str(n) for n in walk] for walk in walks]
    for embedding_size in range(16,528,32):
        for neighborhood_size in [3,5,7,10]:
            model = Word2Vec(str_walks, size=embedding_size, 
                             window=neighborhood_size, min_count=0, 
                             sg=1, workers=2, iter=5)
            model_name = './data/embedding/UAU/'+file.split('/')[-1][:-2]+'_emb_size_'+str(embedding_size)+'_window_'+str(neighborhood_size)+'.bin'
            print(model_name)
            os.makedirs(os.path.dirname(model_name), exist_ok=True)
            model.save(model_name)
# model.wv.vectors.shape  # 128-dimensional vector for each node in the graph

./data/embedding/UAU/walk_length_10_5_per_node_emb_size_16_window_3.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_16_window_5.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_16_window_7.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_16_window_10.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_48_window_3.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_48_window_5.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_48_window_7.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_48_window_10.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_80_window_3.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_80_window_5.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_80_window_7.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_80_window_10.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_112_window_3.bin
./data/embedding/UAU/walk_length_10_5_per_node_emb_size_112_

./data/embedding/UAU/walk_length_10_10_per_node_emb_size_400_window_3.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_400_window_5.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_400_window_7.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_400_window_10.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_432_window_3.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_432_window_5.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_432_window_7.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_432_window_10.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_464_window_3.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_464_window_5.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_464_window_7.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_464_window_10.bin
./data/embedding/UAU/walk_length_10_10_per_node_emb_size_496_window_3.bin
./data/embedding/UAU/walk_length_10

In [136]:
file.split('/')[-1][:-2]

'save'

In [131]:
model.save('model.bin')

In [141]:
model = Word2Vec.load("./data/embedding/UAU/walk_length_10_5_per_node_emb_size_48_window_10.bin")

In [142]:
model.wv['The Beatles']

array([ 0.22050206,  0.12191518, -0.237949  , -0.39075312,  0.1709553 ,
       -0.03249205,  0.44365376,  0.2819415 , -0.32854804,  0.30076352,
       -0.3525293 , -0.09506335, -0.05434151,  0.38181114, -0.00371711,
       -0.05758253,  0.07788604, -0.10289309,  0.17587535,  0.01093263,
       -0.03129449,  0.18775044, -0.24902119, -0.06368333, -0.23091218,
        0.04814355, -0.06474814, -0.21346201,  0.09950143,  0.08116756,
        0.12961952,  0.5916658 ,  0.53450793,  0.10292068, -0.18390879,
        0.5699764 , -0.25688767,  0.1772428 , -0.10273776,  0.05586686,
       -0.03750233, -0.07215051,  0.27854767, -0.00528983, -0.32513002,
       -0.0794755 , -0.13816012,  0.2027588 ], dtype=float32)

In [143]:
model.wv.index2word

['Lady Gaga',
 'The Beatles',
 'Radiohead',
 'Britney Spears',
 'Rihanna',
 'Muse',
 'Paramore',
 'Katy Perry',
 'Coldplay',
 'Madonna',
 'Avril Lavigne',
 'Christina Aguilera',
 'Depeche Mode',
 'Beyoncé',
 'The Killers',
 'Ke$ha',
 'Pink Floyd',
 'Linkin Park',
 'Metallica',
 'Shakira',
 'Michael Jackson',
 'Kylie Minogue',
 'The Cure',
 'Nirvana',
 'Black Eyed Peas',
 'P!nk',
 'Kings of Leon',
 'Queen',
 'Green Day',
 'Iron Maiden',
 'Arctic Monkeys',
 'David Bowie',
 'Placebo',
 'Miley Cyrus',
 'Red Hot Chili Peppers',
 'Evanescence',
 'Lily Allen',
 'Led Zeppelin',
 'Oasis',
 'Taylor Swift',
 'Kelly Clarkson',
 'Glee Cast',
 '30 Seconds to Mars',
 'Mariah Carey',
 'Eminem',
 'System of a Down',
 'Sigur Rós',
 'AC/DC',
 'The Pussycat Dolls',
 'U2',
 'Amy Winehouse',
 'Jennifer Lopez',
 'The Strokes',
 'Björk',
 'Franz Ferdinand',
 'Daft Punk',
 'blink-182',
 "Guns N' Roses",
 'The Smiths',
 'Foo Fighters',
 'Joy Division',
 'My Chemical Romance',
 'The Doors',
 'MGMT',
 'Nine Inch 

In [152]:
model.wv.most_similar('Lindsay Lohan')

[('Ashley Tisdale', 0.9889183044433594),
 ('Ashlee Simpson', 0.9877403974533081),
 ('Nicole Scherzinger', 0.9872066974639893),
 ('Adam Lambert', 0.9841530323028564),
 ('Ke$ha', 0.9829810261726379),
 ('Ciara', 0.9829785227775574),
 ('Christina Aguilera', 0.9822807312011719),
 ('Jennifer Lopez', 0.9818840622901917),
 ('Pixie Lott', 0.9812142848968506),
 ('The Pussycat Dolls', 0.9794191718101501)]

## Metapath2vec using apple2373 implementation

In [85]:
512/16

32.0